In [8]:
from google.colab import drive
drive.mount('/content/drive')

import os

base_path = "/content/drive/MyDrive/Dataset"
quannene_training_path = os.path.join(base_path, "training")
quannene_testting_path = os.path.join(base_path, "testing")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import PIL

MOBILENETV3 LARGE (DONT RUN)

In [46]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, applications
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os
import random
from PIL import Image
from tensorflow.keras.layers import Layer # Moved this import to the top

# -----------------------------
# CONFIGURATION
# -----------------------------
EMBEDDING_DIM = 16
TRIPLET_LOSS_MARGIN = 0.5
IMAGE_SIZE = 224
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
EPOCHS = 8
TRAIN_DATA_PATH = '/content/drive/MyDrive/Dataset/training/dataset'

# -----------------------------
# 1. DATASET: ONLINE TRIPLET GENERATOR
# -----------------------------
class TripletFaceDataset(tf.keras.utils.Sequence):
    """Generate Anchor-Positive-Negative triplets on the fly."""

    def __init__(self, root_dir, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, samples_per_epoch=1000, **kwargs):
        super().__init__(**kwargs) # Modified to pass **kwargs to address UserWarning
        self.root_dir = root_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.samples_per_epoch = samples_per_epoch

        # Map identity -> image paths
        self.identities = []
        self.identity_to_paths = {}
        for person_name in os.listdir(root_dir):
            person_path = os.path.join(root_dir, person_name)
            if os.path.isdir(person_path):
                image_paths = [os.path.join(person_path, f)
                               for f in os.listdir(person_path) # Added 'in os.listdir(person_path)'
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                if len(image_paths) >= 2:
                    self.identities.append(person_name)
                    self.identity_to_paths[person_name] = image_paths

        if len(self.identities) < 2:
            raise ValueError("Need at least 2 identities with 2+ images each.")

    def __len__(self):
        return self.samples_per_epoch // self.batch_size

    def __getitem__(self, idx):
        anchors = []
        positives = []
        negatives = []

        for _ in range(self.batch_size):
            # Anchor identity
            identity_A = random.choice(self.identities)
            A_paths = self.identity_to_paths[identity_A]
            A_path, P_path = random.sample(A_paths, 2)

            # Negative identity
            identity_N = random.choice([i for i in self.identities if i != identity_A])
            N_paths = self.identity_to_paths[identity_N]
            N_path = random.choice(N_paths)

            # Load and preprocess images
            A_img = self.load_image(A_path)
            P_img = self.load_image(P_path)
            N_img = self.load_image(N_path)

            anchors.append(A_img)
            positives.append(P_img)
            negatives.append(N_img)

        return (np.array(anchors), np.array(positives), np.array(negatives)), np.zeros((self.batch_size,))

    def load_image(self, path):
        img = Image.open(path).convert('RGB').resize((self.image_size, self.image_size))
        img_array = np.array(img) / 255.0  # Normalize to [0,1]
        return img_array


# -----------------------------
# 2. BUILD EMBEDDING MODEL
# -----------------------------

@tf.keras.utils.register_keras_serializable()   # <-- FIXED: Changed saving to utils
class L2Normalization(Layer):
    """Custom L2 normalization layer, safe for saving/loading and TFLite."""
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

def build_embedding_model(embedding_dim=16, image_size=224):
    base_model = applications.MobileNetV3Large(
        input_shape=(image_size, image_size, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )

    base_model.trainable = False   # freeze backbone

    x = base_model.output
    x = layers.Dense(embedding_dim)(x)
    x = layers.BatchNormalization()(x)
    x = L2Normalization()(x)       # <-- NO LAMBDA ANYMORE

    model = models.Model(inputs=base_model.input, outputs=x)
    return model

embedding_model = build_embedding_model()
embedding_model.summary()


# -----------------------------
# 3. TRIPLET LOSS
# -----------------------------
def triplet_loss(y_true, y_pred):
    # y_pred shape: (batch*3, embedding_dim) => [A, P, N] flattened
    batch_size = BATCH_SIZE
    embedding_dim = EMBEDDING_DIM

    anchor = y_pred[0:batch_size]
    positive = y_pred[batch_size:batch_size*2]
    negative = y_pred[batch_size*2:batch_size*3]

    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)

    loss = tf.maximum(pos_dist - neg_dist + TRIPLET_LOSS_MARGIN, 0.0)
    return tf.reduce_mean(loss)


# -----------------------------
# 4. TRAINING LOOP
# -----------------------------
dataset = TripletFaceDataset(TRAIN_DATA_PATH, batch_size=BATCH_SIZE)
steps_per_epoch = len(dataset)

# Keras Functional model for triplets
anchor_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
positive_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
negative_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

anchor_embed = embedding_model(anchor_input)
positive_embed = embedding_model(positive_input)
negative_embed = embedding_model(negative_input)

merged_output = layers.Concatenate(axis=0)([anchor_embed, positive_embed, negative_embed])
triplet_model = models.Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)
triplet_model.compile(optimizer=optimizers.Adam(LEARNING_RATE), loss=triplet_loss)

# Train
triplet_model.fit(dataset, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_11        │ (None, 224, 224,  │          0 │ input_layer_22[0… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_11[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_40       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation_40[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_38 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu_38[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation_40[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_39 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_39[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 3,011,792 (11.49 MB)

 Trainable params: 15,408 (60.19 KB)

 Non-trainable params: 2,996,384 (11.43 MB)

Epoch 1/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - loss: 0.5245
Epoch 2/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - loss: 0.3120
Epoch 3/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - loss: 0.2084
Epoch 4/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - loss: 0.1340
Epoch 5/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - loss: 0.1015
Epoch 6/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - loss: 0.0780
Epoch 7/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - loss: 0.0476
Epoch 8/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - loss: 0.0491


MOBILENETV3 SMALL

In [58]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, applications
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import os
import random
from PIL import Image
from tensorflow.keras.layers import Layer # Moved this import to the top

# -----------------------------
# CONFIGURATION
# -----------------------------
EMBEDDING_DIM = 64
TRIPLET_LOSS_MARGIN = 0.5
IMAGE_SIZE = 224
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
EPOCHS = 8
TRAIN_DATA_PATH = '/content/drive/MyDrive/Dataset/training/dataset'

# -----------------------------
# 1. DATASET: ONLINE TRIPLET GENERATOR
# -----------------------------
class TripletFaceDataset(tf.keras.utils.Sequence):
    """Generate Anchor-Positive-Negative triplets on the fly."""

    def __init__(self, root_dir, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, samples_per_epoch=1000, **kwargs):
        super().__init__(**kwargs)
        self.root_dir = root_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.samples_per_epoch = samples_per_epoch

        # Map identity -> image paths
        self.identities = []
        self.identity_to_paths = {}
        for person_name in os.listdir(root_dir):
            person_path = os.path.join(root_dir, person_name)
            if os.path.isdir(person_path):
                image_paths = [os.path.join(person_path, f)
                               for f in os.listdir(person_path)
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                if len(image_paths) >= 2:
                    self.identities.append(person_name)
                    self.identity_to_paths[person_name] = image_paths

        if len(self.identities) < 2:
            raise ValueError("Need at least 2 identities with 2+ images each.")

    def __len__(self):
        return self.samples_per_epoch // self.batch_size

    def __getitem__(self, idx):
        anchors = []
        positives = []
        negatives = []

        for _ in range(self.batch_size):
            # Anchor identity
            identity_A = random.choice(self.identities)
            A_paths = self.identity_to_paths[identity_A]
            A_path, P_path = random.sample(A_paths, 2)

            # Negative identity
            identity_N = random.choice([i for i in self.identities if i != identity_A])
            N_paths = self.identity_to_paths[identity_N]
            N_path = random.choice(N_paths)

            # Load and preprocess images
            A_img = self.load_image(A_path)
            P_img = self.load_image(P_path)
            N_img = self.load_image(N_path)

            anchors.append(A_img)
            positives.append(P_img)
            negatives.append(N_img)

        return (np.array(anchors), np.array(positives), np.array(negatives)), np.zeros((self.batch_size,))

    def load_image(self, path):
        img = Image.open(path).convert('RGB').resize((self.image_size, self.image_size))
        img_array = np.array(img) / 255.0  # Normalize to [0,1]
        return img_array


# -----------------------------
# 2. BUILD EMBEDDING MODEL
# -----------------------------

@tf.keras.utils.register_keras_serializable()
class L2Normalization(Layer):
    """Custom L2 normalization layer, safe for saving/loading and TFLite."""
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

def build_embedding_model(embedding_dim=64, image_size=224):
    base_model = applications.MobileNetV3Small(
        input_shape=(image_size, image_size, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )

    base_model.trainable = False   # freeze backbone

    x = base_model.output
    x = layers.Dense(embedding_dim)(x)
    x = layers.BatchNormalization()(x)
    x = L2Normalization()(x)

    model = models.Model(inputs=base_model.input, outputs=x)
    return model

embedding_model = build_embedding_model()
embedding_model.summary()


# -----------------------------
# 3. TRIPLET LOSS
# -----------------------------
def triplet_loss(y_true, y_pred):
    batch_size = BATCH_SIZE
    embedding_dim = EMBEDDING_DIM

    anchor = y_pred[0:batch_size]
    positive = y_pred[batch_size:batch_size*2]
    negative = y_pred[batch_size*2:batch_size*3]

    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)

    loss = tf.maximum(pos_dist - neg_dist + TRIPLET_LOSS_MARGIN, 0.0)
    return tf.reduce_mean(loss)


# -----------------------------
# 4. TRAINING LOOP
# -----------------------------
dataset = TripletFaceDataset(TRAIN_DATA_PATH, batch_size=BATCH_SIZE)
steps_per_epoch = len(dataset)

# Keras Functional model for triplets
anchor_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
positive_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
negative_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

anchor_embed = embedding_model(anchor_input)
positive_embed = embedding_model(positive_input)
negative_embed = embedding_model(negative_input)

merged_output = layers.Concatenate(axis=0)([anchor_embed, positive_embed, negative_embed])
triplet_model = models.Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)
triplet_model.compile(optimizer=optimizers.Adam(LEARNING_RATE), loss=triplet_loss)

# Train
triplet_model.fit(dataset, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_33      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_15        │ (None, 224, 224,  │          0 │ input_layer_33[0… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_15[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 113, 113,  │          0 │ activation_96[0]… │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_85 (ReLU)     │ (None, 56, 56,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu_85[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_42 (Add)        │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_86 (ReLU)     │ (None, 1, 1, 16)  │          0 │ add_42[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_42         │ (None, 1, 1, 16)  │          0 │ re_lu_86[0][0]    │
│ (Multiply)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 56, 56,    │          0 │ re_lu_85[0][0],   │
│ (Multiply)          │ 16)               │            │ multiply_42[0][0] │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 976,304 (3.72 MB)

 Trainable params: 37,056 (144.75 KB)

 Non-trainable params: 939,248 (3.58 MB)

Epoch 1/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - loss: 0.3637
Epoch 2/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - loss: 0.2317
Epoch 3/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - loss: 0.1207
Epoch 4/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - loss: 0.0949
Epoch 5/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - loss: 0.0742
Epoch 6/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - loss: 0.1071
Epoch 7/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - loss: 0.0680
Epoch 8/8
62/62 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - loss: 0.0459


In [62]:
import os

# Base path
base_path = "/content/drive/MyDrive/Dataset"

# -----------------------------
# 1. Save entire model (architecture + weights)
# -----------------------------
embedding_model_path = os.path.join(base_path, "face_embedding_model.keras")  # Use .keras extension
try:
    embedding_model.save(embedding_model_path)
    print(f"Entire embedding model saved successfully to {embedding_model_path}")
except Exception as e:
    print(f"Error saving entire embedding model: {e}")

# -----------------------------
# 2. Save only weights
# -----------------------------
embedding_weights_path = os.path.join(base_path, "face_embedding.weights.h5")  # Must end with .weights.h5
try:
    embedding_model.save_weights(embedding_weights_path)
    print(f"Embedding model weights saved successfully to {embedding_weights_path}")
except Exception as e:
    print(f"Error saving embedding model weights: {e}")



Entire embedding model saved successfully to /content/drive/MyDrive/Dataset/face_embedding_model.keras
Embedding model weights saved successfully to /content/drive/MyDrive/Dataset/face_embedding.weights.h5


In [4]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Layer # Import Layer for custom class

# Base path
base_path = "/content/drive/MyDrive/Dataset"

# Path to saved embedding model
embedding_model_path = os.path.join(base_path, "face_embedding_model.keras")  # Must match saved file

# Re-define custom layer for loading
@tf.keras.utils.register_keras_serializable()
class L2Normalization(Layer):
    """Custom L2 normalization layer, safe for saving/loading and TFLite."""
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

# Load the model
try:
    embedding_model = tf.keras.models.load_model(embedding_model_path)
    print("Embedding model loaded successfully.")
except Exception as e:
    print(f"Error loading embedding model: {e}")

# Set model to evaluation mode (not necessary in Keras, but good practice)
embedding_model.trainable = False

Embedding model loaded successfully.


In [5]:
embedding_model = tf.keras.models.load_model(embedding_model_path, safe_mode=False)
embedding_model.trainable = False

import tensorflow as tf
import numpy as np
import os
from PIL import Image
from sklearn.metrics.pairwise import euclidean_distances

IMAGE_SIZE = 224

def preprocess_image(img_path):
    """Load image and preprocess exactly like during training."""
    img = Image.open(img_path).convert("RGB")
    img = img.resize((IMAGE_SIZE, IMAGE_SIZE))

    img = np.array(img).astype("float32") / 255.0
    return np.expand_dims(img, axis=0)   # shape: (1, 224, 224, 3)

def get_single_embedding(keras_model, img_path):
    """Returns a 64-D embedding from the Keras embedding model."""
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image not found: {img_path}")

    x = preprocess_image(img_path)

    # Get embedding
    embedding = keras_model.predict(x)[0]  # shape: (64,)
    return embedding

def create_database(keras_model, dataset_path):
    print("\n[INFO] Creating face recognition database...")

    database = {}

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)

        # Ignore hidden/system folders
        if not os.path.isdir(person_path) or person_name.startswith('.'):
            continue

        image_paths = [
            os.path.join(person_path, f)
            for f in os.listdir(person_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]

        if len(image_paths) == 0:
            continue

        print(f"\nProcessing: {person_name}")
        embeddings = []

        for img_path in image_paths:
            try:
                emb = get_single_embedding(keras_model, img_path)
                embeddings.append(emb)
            except Exception as e:
                print(f"  Skipped {img_path} → {e}")

        if len(embeddings) > 0:
            avg_embedding = np.mean(embeddings, axis=0)
            database[person_name] = avg_embedding
            print(f"  ✔ Added {person_name} ({len(embeddings)} images)")
        else:
            print(f"  ✖ No valid embeddings for {person_name}")

    print(f"\n[INFO] Database created: {len(database)} identities.")
    return database

def recognize_single_image(query_embed, database, threshold=1.7):
    best_match = "Unknown"
    min_distance = float("inf")

    for name, ref_embed in database.items():
        distance = euclidean_distances(
            query_embed.reshape(1, -1),
            ref_embed.reshape(1, -1)
        )[0][0]

        if distance < min_distance:
            min_distance = distance
            best_match = name

    if min_distance < threshold:
        return best_match, min_distance
    else:
        return "Unknown", min_distance

TRAIN_DATA_PATH = "/content/drive/MyDrive/Dataset/training/dataset"
QUERY_IMAGE_PATH = "/content/drive/MyDrive/Dataset/testing/Screenshot 2025-12-08 135312.png"
RECOGNITION_THRESHOLD = 1.7

# Load model
embedding_model = tf.keras.models.load_model(
    "/content/drive/MyDrive/Dataset/face_embedding_model.keras",
    safe_mode=False
)
embedding_model.trainable = False

# Create database
face_database = create_database(embedding_model, TRAIN_DATA_PATH)
people = list(face_database.keys())

# Compute query embedding
query_embedding = get_single_embedding(embedding_model, QUERY_IMAGE_PATH)

# Identify
identity, distance = recognize_single_image(
    query_embedding, face_database, threshold=RECOGNITION_THRESHOLD
)

print("\n==============================")
print(f"Query Image: {os.path.basename(QUERY_IMAGE_PATH)}")
print(f"Identity: {identity}")
print(f"Distance: {distance:.4f}")
print("==============================")


[INFO] Creating face recognition database...

Processing: MrVinh
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
  ✔ Added MrVinh (2 images)

Processing: MrThanh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
  ✔ Added MrThanh (4 images)

Processing: MrNam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
  ✔ Added MrNam (11 images)

Processing: MrDuc
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
  ✔ Added MrDuc (2 images)

Processing: MrCan
1/1 ━━━━━━━━━━━━━━━━━━━━

In [11]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image

# Config
TRAIN_DATA_PATH = '/content/drive/MyDrive/Dataset/training/dataset'
IMAGE_SIZE = 224
EMBEDDING_DIM = embedding_model.output_shape[-1]  # 64
BATCH_SIZE = 16

# Prepare embeddings
people = [d for d in os.listdir(TRAIN_DATA_PATH) if os.path.isdir(os.path.join(TRAIN_DATA_PATH,d)) and not d.startswith('.')]
num_people = len(people)

X = []  # embeddings
y = []  # labels

for idx, person in enumerate(people):
    folder = os.path.join(TRAIN_DATA_PATH, person)
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder, fname)
            img = Image.open(img_path).convert('RGB').resize((IMAGE_SIZE, IMAGE_SIZE))
            arr = np.array(img)/255.0
            arr = np.expand_dims(arr, axis=0).astype(np.float32)
            emb = embedding_model.predict(arr)
            X.append(emb[0])
            y.append(idx)

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

print("Embedding matrix:", X.shape)
print("Labels:", y.shape)

input_emb = tf.keras.Input(shape=(EMBEDDING_DIM,))
x = tf.keras.layers.Dense(64, activation='relu')(input_emb)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(num_people, activation='softmax', dtype='float32')(x)

classifier_model = tf.keras.Model(inputs=input_emb, outputs=output)
classifier_model.compile(optimizer='adam',
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])

classifier_model.summary()
classifier_model.fit(X, y, epochs=50, batch_size=8, verbose=1)

# Input image (0..255)
input_image_raw = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), dtype=tf.float32, name="image_input_raw")

# Rescale to 0..1
x = tf.keras.layers.Rescaling(1./255.0)(input_image_raw)

# Embedding
embedding_output = embedding_model(x)

# L2 normalization
embedding_norm = tf.keras.layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(embedding_output)

# Identity prediction
identity_output = classifier_model(embedding_norm)

identity_model_final = tf.keras.Model(inputs=input_image_raw, outputs=identity_output)
identity_model_final.summary()

def predict_identity_raw(image_path, model, people):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    arr = tf.keras.preprocessing.image.img_to_array(img)
    arr = np.expand_dims(arr.astype(np.float32), axis=0)
    preds = model.predict(arr)
    idx = np.argmax(preds)
    return people[idx], float(preds[0][idx])

test_image = '/content/drive/MyDrive/Dataset/testing/PXL_20250609_141755633.jpg'
name, prob = predict_identity_raw(test_image, identity_model_final, people)
print("Predicted:", name, prob)

save_path = "/content/drive/MyDrive/Dataset/face_classifier.keras"
identity_model_final.save(save_path)
print("Saved classifier to:", save_path)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,936 (19.28 KB)

 Trainable params: 4,808 (18.78 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input_raw (InputLayer)    │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_23 (Functional)      │ (None, 64)             │       976,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional (Functional)         │ (None, 8)              │         4,936 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 981,240 (3.74 MB)

 Trainable params: 4,808 (18.78 KB)

 Non-trainable params: 976,432 (3.72 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted: MrSheeraz 0.748803436756134
Saved classifier to: /content/drive/MyDrive/Dataset/face_classifier.keras


In [12]:
import os

def get_file_size_mb(path):
    if os.path.exists(path):
        size_bytes = os.path.getsize(path)
        size_mb = size_bytes / (1024 * 1024)
        print(f"✔ File size: {size_mb:.2f} MB")
    else:
        print(f"❌ File not found: {path}")

embedding_path = "/content/drive/MyDrive/Dataset/face_embedding_model.keras"
classifier_path = "/content/drive/MyDrive/Dataset/face_classifier.keras"

get_file_size_mb(embedding_path)
get_file_size_mb(classifier_path)


✔ File size: 4.26 MB
✔ File size: 4.35 MB


**PRUNING + INT8 QUANTIZATION TFLITE**

In [13]:
import os
import tensorflow as tf
import numpy as np

# Assuming these variables are defined in your execution environment:
# embedding_model (MobileNetV3 backbone)
# classifier_model (64-D Linear head)
# people (list of identities)
# quannene_training_path (path to training dataset root)

# ==============================
# USER SETTINGS
# ==============================
BASE_PATH = "/content/drive/MyDrive/Dataset"
OUT_DIR = os.path.join(BASE_PATH, "tflite_int8_pruned_small")
os.makedirs(OUT_DIR, exist_ok=True)

# 💡 CHANGE 1: Increase sparsity for more aggressive size reduction.
# Target 75% zero weights across the model.
FINAL_SPARSITY = 0.5
FINETUNE_EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = (224, 224)

# ==============================
# 1. PREPARE DATASET (Normalization Change)
# ==============================
def load_img(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)

    # 💡 CHANGE 2: Normalize to the [-1, 1] range.
    # This is standard practice for MobileNetV3 and improves INT8 quantization ranges.
    img = (img / 127.5) - 1.0

    return img, label

dataset_root = os.path.join(quannene_training_path, "dataset")
train_files, train_labels, val_files, val_labels = [], [], [], []

for label_idx, person in enumerate(people):
    pdir = os.path.join(dataset_root, person)
    files = sorted([os.path.join(pdir,f) for f in os.listdir(pdir)
                    if f.lower().endswith(('.jpg','.png','.jpeg'))])
    if len(files)==0: continue

    split = max(1, int(0.8*len(files)))
    train_files += files[:split]
    train_labels += [label_idx]*split

    val = files[split:]
    if len(val)==0:
        val = files[-1:]
    val_files += val
    val_labels += [label_idx]*len(val)

train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
val_ds   = tf.data.Dataset.from_tensor_slices((val_files, val_labels))

train_ds = train_ds.map(load_img, num_parallel_calls=tf.data.AUTOTUNE)\
                   .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.map(load_img, num_parallel_calls=tf.data.AUTOTUNE)\
                 .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"[INFO] Train: {len(train_files)}  Val: {len(val_files)}")

# ==============================
# 2. AGGRESSIVE MANUAL PRUNING (Targets ALL Conv Kernels)
# ==============================
def manual_prune_all(model, final_sparsity):
    """Applies magnitude pruning across all 'kernel' weights in the model."""
    clone = tf.keras.models.clone_model(model)
    clone.set_weights(model.get_weights())

    all_weights, refs = [], []

    # 1. Collect all kernel weights (from embedding model and classifier)
    for li, layer in enumerate(clone.layers):
        for vi, w in enumerate(layer.weights):
            # Check for convolution or linear layer weights
            if "kernel" in w.name.lower() or "dense" in w.name.lower():
                wflat = w.numpy().flatten()
                all_weights.append(wflat)
                # refs store index, weight index, and original shape
                refs.append((li, vi, w.numpy().shape))

    if len(all_weights) == 0:
        print("[WARN] No kernel weights found for pruning.")
        return clone

    # 2. Determine global pruning threshold
    concat = np.concatenate(all_weights)
    k = int(final_sparsity * concat.size)
    thresh = np.sort(np.abs(concat))[k-1]

    # 3. Apply zeros based on threshold
    pos = 0
    for (li, vi, orig_shape), wflat in zip(refs, all_weights):
        n = wflat.size
        # Get the segment from the global concatenated weights
        seg = concat[pos:pos+n]

        # Prune: set values less than thresh to 0.0
        pruned = np.where(np.abs(seg) < thresh, 0.0, seg).reshape(orig_shape)

        # Update the layer weights
        layer = clone.layers[li]
        weights = layer.get_weights()
        weights[vi] = pruned
        layer.set_weights(weights)
        pos += n

    print(f"[INFO] Aggressive Manual Pruning applied (Sparsity: {final_sparsity*100:.0f}%)")
    return clone

# Prune the entire model (both embedding_model and classifier)
# ⚠️ This requires access to the full model, which the original code assumes is 'classifier_model'
# I assume your 'classifier_model' is the small head, and 'embedding_model' is the backbone.
# We will combine them for pruning and then reconstruct.

# Since we cannot easily combine and re-split for pruning, we will prune the embedding model and classifier separately:

pruned_embedding_model = manual_prune_all(embedding_model, FINAL_SPARSITY)
pruned_classifier = manual_prune_all(classifier_model, FINAL_SPARSITY)


# ==============================
# 3. BUILD FULL MODEL WITH PRUNED COMPONENTS
# ==============================
def build_full_pruned(FRmodel, classifier_model):
    # FRmodel is now the pruned embedding model
    FRmodel.trainable = False

    inp = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32)

    # 💡 CHANGE 3: Remove Rescaling layer, as normalization is now done in load_img

    e = FRmodel(inp) # Pass input directly to the pruned embedding model
    if isinstance(e, dict):
        e = list(e.values())[0]
    if len(e.shape) > 2:
        e = tf.keras.layers.Flatten()(e)

    # Replace Lambda with serializable custom L2Norm
    # 💡 L2Norm can sometimes be tricky for quantization, but we keep it for now.
    e = tf.keras.layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1), name="l2norm")(e)

    out = classifier_model(e) # classifier_model is now the pruned classifier head
    return tf.keras.Model(inputs=inp, outputs=out, name="int8_pruned_model")

final_model = build_full_pruned(pruned_embedding_model, pruned_classifier)
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

# ==============================
# 4. FINE-TUNE (Retraining the pruned weights)
# ==============================
print("[INFO] Fine-tuning aggressively pruned model...")
# Fine-tuning is critical to recover accuracy after pruning
final_model.fit(train_ds, validation_data=val_ds, epochs=FINETUNE_EPOCHS)
print("[INFO] Fine-tuning complete")

import os
import tensorflow as tf
import numpy as np
# ... (All previous imports)

# ... (Sections 1, 2, 3, 4 remain as in the previous aggressive pruning script) ...

# ==============================
# 5. INT8 FULL-INTEGER QUANTIZATION (Optimized for Minimum Size)
# ==============================
def representative_dataset():
    # Only need a few samples for the converter to calculate quantization ranges
    for f in train_files[:200]:
        img = tf.io.read_file(f)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.resize(img, IMG_SIZE)
        img = tf.cast(img, tf.float32)
        # Apply the same normalization as in load_img: [-1, 1]
        img = (img / 127.5) - 1.0
        img = tf.expand_dims(img,0)
        yield [img]

# --- CRITICAL CHANGE: Use INT8 Input/Output for smallest possible file size ---
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# ⚠️ FORCING INT8 INPUT/OUTPUT REDUCES FILE SIZE SIGNIFICANTLY
# This forces the final TFLite file to only contain INT8 tensors,
# which are 4x smaller than FLOAT32 weights.
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_int8 = converter.convert()
tflite_path = os.path.join(OUT_DIR, "identity_pruned_int8_smallest.tflite")

# --- Manual stripping of TFLite (Optional but helps) ---
# Forcing the full INT8 conversion usually takes care of the size,
# as the vast majority of the file is now 1-byte integers instead of 4-byte floats.

with open(tflite_path, "wb") as f:
    f.write(tflite_int8)

print("[SAVED] INT8 model ->", tflite_path)
final_size_mb = os.path.getsize(tflite_path) / (1024*1024)
print(f"[SIZE] Final TFLite size: {final_size_mb:.2f} MB")

# --- FINAL ANALYSIS ---
if final_size_mb <= 2.5:
    print("[SUCCESS] Target size achieved!")
else:
    print(f"[NOTE] Conversion complete. If the size is still > 2.5MB, the limiting factor is the dense complexity of the MobileNetV3 backbone.")

print("[DONE] Pipeline finished successfully.")

[INFO] Train: 33  Val: 13
[INFO] Aggressive Manual Pruning applied (Sparsity: 50%)
[INFO] Aggressive Manual Pruning applied (Sparsity: 50%)
[INFO] Fine-tuning aggressively pruned model...
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.3596 - loss: 1.7823 - val_accuracy: 0.2308 - val_loss: 2.1159
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 899ms/step - accuracy: 0.3596 - loss: 1.7923 - val_accuracy: 0.2308 - val_loss: 2.1096
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 992ms/step - accuracy: 0.3596 - loss: 1.7874 - val_accuracy: 0.2308 - val_loss: 2.1035
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 902ms/step - accuracy: 0.3596 - loss: 1.8131 - val_accuracy: 0.2308 - val_loss: 2.0975
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.3596 - loss: 1.8062 - val_accuracy: 0.2308 - val_loss: 2.0917
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.3596 - loss: 1.7901 - val_accuracy: 0.2308 - val_loss: 2.0860
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 955ms/step - accuracy: 0.

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


[SAVED] INT8 model -> /content/drive/MyDrive/Dataset/tflite_int8_pruned_small/identity_pruned_int8_smallest.tflite
[SIZE] Final TFLite size: 1.20 MB
[SUCCESS] Target size achieved!
[DONE] Pipeline finished successfully.
